In [ ]:
import os
import pandas as pd
import bnlearn as bn
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
from utils_behaviour import get_model, draw_bn_plotly, cpd_add_n, filter_cpd_for_hotspot, get_outlier
from config import category_value_map, feature_name_map, select_group_behaviour

In [ ]:
# This is from TDAv2
combined_data = pd.read_csv('../ComputedData/ForModel/combined_data_with_hotspot.csv')
combined_data.shape

In [ ]:
data = combined_data[select_group_behaviour].copy()
data['facility'] = data[['youbike_100m_count']].apply(lambda row: '1' if (row > 0).any() else '0', axis=1)
data.drop(columns=['youbike_100m_count'], inplace=True)

max_speed = data['速限-第1當事者'].max()
bins = range(0, int(max_speed) + 11, 10)

data['速限-第1當事者'] = pd.cut(
    data['速限-第1當事者'],
    bins=bins,
    right=False, 
    include_lowest=True,
    labels=[f"{i}-{i+9}" for i in bins[:-1]]
)

data0 = data[data['facility'] == '0']
data1 = data[data['facility'] == '1']

In [ ]:
parent = [
    '號誌-號誌種類名稱', '號誌-號誌動作名稱','車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱',
    '車道劃分設施-分道設施-路面邊線名稱', '車道劃分設施-分向設施子類別名稱', '道路型態子類別名稱',
    '速限-第1當事者', '道路類別-第1當事者-名稱', 'facility', 'COUNTYNAME'
    ]
cause = ['肇因研判子類別名稱-主要']
result = ['事故類型及型態子類別名稱']

white_list = [
    ('速限-第1當事者', '肇因研判子類別名稱-主要'),
    ('道路類別-第1當事者-名稱', '肇因研判子類別名稱-主要'),
    ('道路型態子類別名稱', '肇因研判子類別名稱-主要'),
    ('facility', '肇因研判子類別名稱-主要'),
    ('號誌-號誌種類名稱', '肇因研判子類別名稱-主要'),
    # ('肇因研判子類別名稱-主要', '事故類型及型態子類別名稱'),
]

black_list = []
# cause -> parent
black_list += [(c, p) for c in cause for p in parent]
# result -> parent/cause
black_list += [(r, x) for r in result for x in (parent + cause + result)]
# 保險：parent -> result（避免直接 shortcut，如果只想透過肇因解釋）
# black_list += [(p, r) for p in parent for r in result]

In [ ]:
model0, model_param0, model_independence0 = get_model(data0)
model1, model_param1, model_independence1 = get_model(data1)
model_all, model_param_all, model_independence_all = get_model(data)

In [ ]:
draw_bn_plotly(model_independence_all, layout_algo='spring', en=False, width=600, height=400, seed=42, iter=30)

## Inference
這個方法針對特定的推論得出cpt，現在討論反向所以evidence會是肇因，討論特定特徵下不同設計的機率

In [ ]:
parent = ['速限-第1當事者', '道路類別-第1當事者-名稱', '道路型態子類別名稱', 'facility', '號誌-號誌種類名稱']
child = '肇因研判子類別名稱-主要'
evidence_v = list(data['肇因研判子類別名稱-主要'].value_counts().head(5).index)

In [ ]:
for state, model_pm in enumerate ([model_param0, model_param1]):
    dt = data0 if state == 0 else data1
    for v in evidence_v:
        q2 = bn.inference.fit(
            model_pm, 
            variables=parent,
            evidence={'肇因研判子類別名稱-主要': v})

        model_df = q2.df
        evidence_df = dt[dt['肇因研判子類別名稱-主要'] == v]
        filtered_condition = cpd_add_n(parent, child, model_df, evidence_df, cpd=False, threshold=0)
        final_filtered = filtered_condition[['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p', 'n', '道路類別-第1當事者-名稱']]
        final_filtered = final_filtered.sort_values(by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'], ascending=True)

        final_filtered.to_csv(f'../ComputedData/Behaviour_split/{state}_{v}.csv', encoding='utf-8', index=False)

### Full
分別存每個肇因的離群

In [ ]:
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱']

for i in range(len(evidence_v)):
    condition_0 = pd.read_csv(f'../ComputedData/Behaviour_split/0_{evidence_v[i]}.csv', encoding='utf-8')
    condition_1 = pd.read_csv(f'../ComputedData/Behaviour_split/1_{evidence_v[i]}.csv', encoding='utf-8')
    condition_concat = pd.concat([condition_0, condition_1], axis=0)
    # All filters
    hotspot = filter_cpd_for_hotspot(condition_concat)
    final_filtered = hotspot[(hotspot['p'] >= 0.05) & (hotspot['n'] >= 100)]
    # Filters end
    final = final_filtered.sort_values(
        by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'],
        ascending=True)
    # EN version
    for col in parent_filtered:
        final[col] = final[col].map(category_value_map[col])
    final.rename(columns=feature_name_map, inplace=True)
    final.to_excel(f'../ComputedData/Behaviour_split/full_{evidence_v[i]}.xlsx', encoding='utf-8', index=False)

### Outlier Analysis
存前五大常出現肇因中離群

In [ ]:
outlier_data = pd.DataFrame()
fac = 1
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱', '肇因研判子類別名稱-主要']

for i in range(len(evidence_v)):
    try:
        original_condition = pd.read_csv(f'../ComputedData/Behaviour_split/{fac}_{evidence_v[i]}.csv', encoding='utf-8')
        # origin是用來計算離群，所以不能對他篩選，應該要篩選outlier_data
        new_condition = filter_cpd_for_hotspot(original_condition)
        new_condition_out = get_outlier(original_condition, new_condition)
        new_condition_out['肇因研判子類別名稱-主要'] = evidence_v[i]
        outlier_data = pd.concat([outlier_data, new_condition_out], axis=0)
        outlier_data = outlier_data[outlier_data['n'] >= 50]
        # 由於outlier太多，增加更高的threshold
        # outlier_data = outlier_data[outlier_data['p'] >= 0.03].nlargest(20, 'p')
        threshold = outlier_data['p'].quantile(0.5) 
        outlier_data = outlier_data[outlier_data['p'] >= threshold]

        outlier_data = outlier_data[['肇因研判子類別名稱-主要', '號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p', 'n']]
        outlier_data = outlier_data.sort_values(by=['肇因研判子類別名稱-主要', '號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'p'], ascending=True)
    except Exception as e:
        print(f"Error processing {evidence_v[i]}: {e}")

# en version
for col in parent_filtered:
    outlier_data[col] = outlier_data[col].map(category_value_map[col])
outlier_data.rename(columns=feature_name_map, inplace=True)
outlier_data.to_excel(f'../ComputedData/Behaviour_split/outlier_{fac}.xlsx', index=False)

### Hotspot Ratio Analysis

In [ ]:
def make_df(prefix, ev):
    data_sum = {}
    for i in range(len(ev)):
        try:
            condition = pd.read_csv(f'../ComputedData/Behaviour/{prefix}_{ev[i]}.csv', encoding='utf-8')
            data = filter_cpd_for_hotspot(condition)
            data_sum[ev[i]] = round(data['p'].sum(), 4)
        except Exception as e:
            print(f"Error processing {ev[i]}: {e}")

    df_sum = pd.DataFrame(
        list(data_sum.items()), 
        columns=['肇因研判子類別名稱-主要', 'p']
    )
    df_sum['facility'] = prefix
    return df_sum

df0 = make_df("0", evidence_v)
df1 = make_df("1", evidence_v)
final_df = pd.concat([df0, df1], ignore_index=True)

final_df.sort_values(by=['肇因研判子類別名稱-主要'], ascending=True, inplace=True)
final_df = final_df[['肇因研判子類別名稱-主要', 'facility', 'p']]

for col in ['肇因研判子類別名稱-主要']:
    final_df[col] = final_df[col].map(category_value_map[col])
final_df.rename(columns=feature_name_map, inplace=True)

final_df.to_excel(f'../ComputedData/Behaviour_split/final_sum.xlsx', index=False)

## This is for original data
1. 分成原始資料做的原因在於分開比較可能因為道路組合數量不同造成整體機率下降，這樣會導致在有無設施下，相同組合下有設施不任何顯著組合都是高於無設施的，因為無設施組合更多。<br/>
2. 但使用原始資料進行分析不能單純依照p和n去篩選，不然剩下的資料都會是無設施，因為有設施資料量本來就比較少。<br/>
> 為了防止上述偏差，獲取outlier之後分開設施1和0並取前10大常見組合，n也需要設置，因為小樣本偏差還是應該去除
> 問題：最終比較的還是p

In [ ]:
parent = ['速限-第1當事者', '道路類別-第1當事者-名稱', '道路型態子類別名稱', 'facility', '號誌-號誌種類名稱']
child = '肇因研判子類別名稱-主要'

for v in evidence_v:
    q2 = bn.inference.fit(
        model_param_all, 
        variables=parent,
        evidence={'肇因研判子類別名稱-主要': v})

    model_df = q2.df
    evidence_df = data[data['肇因研判子類別名稱-主要'] == v]
    filtered_condition = cpd_add_n(parent, child, model_df, evidence_df, cpd=False, threshold=-1)
    final_filtered = filtered_condition[['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p', 'n', '道路類別-第1當事者-名稱']]
    final_filtered = final_filtered.sort_values(by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'], ascending=True)

    final_filtered.to_csv(f'../ComputedData/Behaviour/all_{v}.csv', encoding='utf-8', index=False)

In [ ]:
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱']

for i in range(len(evidence_v)):
    condition = pd.read_csv(f'../ComputedData/Behaviour/all_{evidence_v[i]}.csv', encoding='utf-8')
    # All filters
    hotspot = filter_cpd_for_hotspot(condition)
    final_filtered = hotspot[(hotspot['p'] >= 0.03) & (hotspot['n'] >= 30)]
    # Filters end
    final = final_filtered.sort_values(
        by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'],
        ascending=True)
    # EN version
    for col in parent_filtered:
        final[col] = final[col].map(category_value_map[col])
    final.rename(columns=feature_name_map, inplace=True)
    final.to_excel(f'../ComputedData/Behaviour/full_Origin_{evidence_v[i]}.xlsx', encoding='utf-8', index=False)

### Outlier analysis for original data

In [ ]:
outlier_data = pd.DataFrame()
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱', '肇因研判子類別名稱-主要']

for i in range(len(evidence_v)):
    original_data = pd.read_csv(f'../ComputedData/Behaviour/all_{evidence_v[i]}.csv', encoding='utf-8')
    # origin是用來計算離群，所以不能對他篩選，應該要篩選outlier_data
    new_condition = filter_cpd_for_hotspot(original_data)
    outlier = get_outlier(original_data, new_condition)
    outlier['肇因研判子類別名稱-主要'] = evidence_v[i]
    outlier_data = pd.concat([outlier_data, outlier], axis=0)

filtered_0 = outlier_data[(outlier_data['facility'] == 0) &
                          (outlier_data['n'] > 30)].sort_values(by=['p'], ascending=False).head(25)
filtered_1 = outlier_data[(outlier_data['facility'] == 1) &
                          (outlier_data['n'] > 30)].sort_values(by=['p'], ascending=False).head(25)
full_filter = pd.concat([filtered_0, filtered_1], axis=0)

for col in parent_filtered:
    full_filter[col] = full_filter[col].map(category_value_map[col])
full_filter.rename(columns=feature_name_map, inplace=True)
full_filter.to_excel(f'../ComputedData/Behaviour/Origin_outlier.xlsx', index=False)

In [ ]:
def make_df(ev, fac):
    data_sum = {}
    for i in range(len(ev)):
        condition = pd.read_csv(f'../ComputedData/Behaviour/all_{ev[i]}.csv', encoding='utf-8')
        condition = condition[condition['facility'] == fac]
        data = filter_cpd_for_hotspot(condition)
        data_sum[ev[i]] = round(data['p'].sum(), 4)

    df_sum = pd.DataFrame(
        list(data_sum.items()), 
        columns=['肇因研判子類別名稱-主要', 'p']
    )
    return df_sum

df_0 = make_df(evidence_v, fac=0)
df_1 = make_df(evidence_v, fac=1)
df_0['facility'] = 0
df_1['facility'] = 1
df = pd.concat([df_0, df_1], ignore_index=True)
df.sort_values(by=['肇因研判子類別名稱-主要'], ascending=True, inplace=True)

for col in ['肇因研判子類別名稱-主要']:
    df[col] = df[col].map(category_value_map[col])
df.rename(columns=feature_name_map, inplace=True)
df.to_excel(f'../ComputedData/Behaviour/final_sum.xlsx', index=False)

## CPD
沒有針對特徵，回傳肇因的因

In [ ]:
CPDs = bn.print_CPD(model_param_all)
dfprob_cause = CPDs[child]
filtered = cpd_add_n(parent, child, dfprob_cause, data, threshold=0)

In [ ]:
filtered_n = filtered[(filtered['p'] > 0.1) & (filtered['n'] > 100)]
filtered_n['facility'] = filtered_n['facility'].astype(int)
final_filtered = filter_cpd_for_hotspot(filtered_n)
final_filtered = final_filtered.nlargest(50, 'p')
final_filtered = final_filtered[['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', '肇因研判子類別名稱-主要', 'p', 'n']]
final_filtered = final_filtered.sort_values(by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'], ascending=True)
# EN version
for col in ['道路型態子類別名稱', '號誌-號誌種類名稱', '肇因研判子類別名稱-主要']:
    final_filtered[col] = final_filtered[col].map(category_value_map[col])
final_filtered.rename(columns=feature_name_map, inplace=True)
final_filtered

In [ ]:
from openpyxl import load_workbook

path = "../BNtables/final_filtered_en.xlsx"
final_filtered.to_excel(path, index=False)
wb = load_workbook(path)
ws = wb.active

for i in range(1, 4):
    col_idx = i  # 1st column
    merge_start = 2  # start from row 2 (header is in row 1)
    current_value = ws.cell(row=merge_start, column=col_idx).value

    for row in range(merge_start + 1, ws.max_row + 2):  # +2 是為了最後一個 flush
        value = ws.cell(row=row, column=col_idx).value if row <= ws.max_row else None

        if value != current_value:
            # 合併區間
            if row - merge_start > 1:
                ws.merge_cells(start_row=merge_start, start_column=col_idx,
                            end_row=row-1, end_column=col_idx)
                # 垂直置中
                ws.cell(row=merge_start, column=col_idx).alignment = ws.cell(row=merge_start, column=col_idx).alignment.copy(vertical="center", horizontal="center")
            # 更新起點
            merge_start = row
            current_value = value

    wb.save(path)


### 確認CPT的機率

In [ ]:
import numpy as np
# parent 組合數
q = dfprob_cause.groupby(parent, dropna=False).ngroups
assert np.prod([data[col].nunique() for col in parent])==q
# 每個 parent 組合底下的機率和都應該 ≈ 1
chk = dfprob_cause.groupby(parent, dropna=False)['p'].sum().unique()